In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [1]:
!ls

adc.json  drive  sample_data


In [2]:
cd drive/Houston/

/content/drive/Houston


In [0]:
import numpy as np
import cv2

In [0]:
hsi = []
for i in range(3):
  img_path = "pca/hsi_pc" + str(i) + ".png"
  img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
  hsi.append(img)
        
hsi = np.array(hsi)

lidar = cv2.imread("pca/lidar.png", cv2.IMREAD_GRAYSCALE)
lidar = lidar.reshape(1, lidar.shape[0], lidar.shape[1])

In [5]:
print(hsi.shape)
print(lidar.shape)

(3, 349, 1905)
(1, 349, 1905)


In [0]:
train_file_name = "labels/train.txt"
test_file_name = "labels/test.txt"

In [0]:
file = open(train_file_name)
triplets = file.read().split()
for i in range(0, len(triplets)):
    triplets[i] = triplets[i].split(",")
train_array = np.array(triplets, dtype=int)
file.close()


file = open(test_file_name)
triplets = file.read().split()
for i in range(0, len(triplets)):
    triplets[i] = triplets[i].split(",")
test_array = np.array(triplets, dtype=int)
file.close()

HEIGHT = train_array.shape[0]
WIDTH = train_array.shape[1]

In [0]:
hsi_train_data = []
hsi_test_data = []
lidar_train_data = []
lidar_test_data = []
train_labels = []
test_labels = []
for i in range(HEIGHT):
    for j in range(WIDTH):
        if train_array[i, j] != 0:
            hsi_train_data.append(hsi[:, i, j])
            lidar_train_data.append(lidar[:, i, j])
            train_labels.append(train_array[i, j])
        if test_array[i,j] != 0:
            hsi_test_data.append(hsi[:, i, j])
            lidar_test_data.append(lidar[:, i, j])
            test_labels.append(test_array[i, j])

In [0]:
hsi_train_data = np.array(hsi_train_data)
lidar_train_data = np.array(lidar_train_data)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [10]:
print(hsi_train_data.shape)
print(lidar_train_data.shape)
print(train_labels.shape)
print(test_labels.shape)

(2832, 3)
(2832, 1)
(2832,)
(12197,)


In [11]:
import keras

train_one_hot = keras.utils.to_categorical(train_labels-1)
test_one_hot = keras.utils.to_categorical(test_labels-1)

Using TensorFlow backend.


In [0]:
HSI_PATCH_SIZE = 27
LiDAR_PATCH_SIZE = 41
CONV1 = 500
CONV2 = 100
FC1 = 200
FC2 = 84
LEARNING_RATE = 0.005

In [13]:
padded_hsi = np.lib.pad(hsi, ((0,0), (HSI_PATCH_SIZE//2, HSI_PATCH_SIZE//2), (HSI_PATCH_SIZE//2,HSI_PATCH_SIZE//2)), 'reflect')
padded_lidar = np.lib.pad(lidar, ((0,0), (LiDAR_PATCH_SIZE//2, LiDAR_PATCH_SIZE//2), (LiDAR_PATCH_SIZE//2,LiDAR_PATCH_SIZE//2)), 'reflect')
print(padded_hsi.shape)
print(padded_lidar.shape)

(3, 375, 1931)
(1, 389, 1945)


In [0]:
def get_patches(data, patch_size, row, column):
  offset = patch_size // 2
  row_low = row - offset
  row_high = row + offset
  col_low = column - offset
  col_high = column + offset
  return data[0:, row_low:row_high + 1, col_low:col_high + 1].reshape(patch_size, patch_size, data.shape[0])

In [0]:
hsi_train_patches = []
hsi_test_patches = []
lidar_train_patches = []
lidar_test_patches = []
for i in range(HEIGHT):
  for j in range(WIDTH):
    if train_array[i, j] != 0:
      hsi_train_patches.append(get_patches(padded_hsi, HSI_PATCH_SIZE, i+HSI_PATCH_SIZE//2, j+HSI_PATCH_SIZE//2))
      lidar_train_patches.append(get_patches(padded_lidar, LiDAR_PATCH_SIZE, i+LiDAR_PATCH_SIZE//2, j+LiDAR_PATCH_SIZE//2))
    if test_array[i, j] != 0:
      hsi_test_patches.append(get_patches(padded_hsi, HSI_PATCH_SIZE, i+HSI_PATCH_SIZE//2, j+HSI_PATCH_SIZE//2))
      lidar_test_patches.append(get_patches(padded_lidar, LiDAR_PATCH_SIZE, i+LiDAR_PATCH_SIZE//2, j+LiDAR_PATCH_SIZE//2))

In [16]:
hsi_train_patches = np.array(hsi_train_patches)
hsi_test_patches = np.array(hsi_test_patches)
lidar_train_patches = np.array(lidar_train_patches)
lidar_test_patches = np.array(lidar_test_patches)
print(hsi_train_patches.shape)
print(hsi_test_patches.shape)
print(lidar_train_patches.shape)
print(lidar_test_patches.shape)

(2832, 27, 27, 3)
(12197, 27, 27, 3)
(2832, 41, 41, 1)
(12197, 41, 41, 1)


In [0]:
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras.layers import InputLayer, Input
from tensorflow.python.keras.layers import  MaxPooling2D
from tensorflow.python.keras.layers import BatchNormalization, Dropout
from tensorflow.python.keras.optimizers import Adam,SGD

BANDS = hsi_train_patches.shape[3]
NUM_CLS = train_one_hot.shape[1]


hsi_input = Input(shape=(HSI_PATCH_SIZE, HSI_PATCH_SIZE, BANDS))

hsi_conv1 = Conv2D(kernel_size=6, strides=2, filters=CONV1, padding='same', activation='relu', name='hsi_conv1')(hsi_input)
hsi_batch1 = BatchNormalization()(hsi_conv1)
hsi_max_pool1 = MaxPooling2D(pool_size=2, strides=2)(hsi_batch1)
hsi_conv2 = Conv2D(kernel_size=5, strides=2, filters=CONV2, padding='same', activation='relu', name='hsi_conv2')(hsi_max_pool1)
hsi_batch2 = BatchNormalization()(hsi_conv2)
hsi_max_pool2 = MaxPooling2D(pool_size=2, strides=2)(hsi_batch2)

hsi_flatten = Flatten()(hsi_max_pool2)

In [0]:
from tensorflow.python.keras.models import Sequential,Model
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras.layers import InputLayer, Input, concatenate
from tensorflow.python.keras.layers import  MaxPooling2D
from tensorflow.python.keras.layers import BatchNormalization, Dropout
from tensorflow.python.keras.optimizers import Adam,SGD

BANDS = lidar_train_patches.shape[3]
NUM_CLS = train_one_hot.shape[1]


lidar_input = Input(shape=(LiDAR_PATCH_SIZE, LiDAR_PATCH_SIZE, BANDS))

lidar_conv1 = Conv2D(kernel_size=6, strides=2, filters=CONV1, padding='same', activation='relu', name='lidar_conv1')(lidar_input)
lidar_batch1 = BatchNormalization()(lidar_conv1)
lidar_max_pool1 = MaxPooling2D(pool_size=2, strides=2)(lidar_batch1)
lidar_conv2 = Conv2D(kernel_size=5, strides=2, filters=CONV2, padding='same', activation='relu', name='lidar_conv2')(lidar_max_pool1)
lidar_batch2 = BatchNormalization()(lidar_conv2)
lidar_max_pool2 = MaxPooling2D(pool_size=2, strides=2)(lidar_batch2)

lidar_flatten = Flatten()(lidar_max_pool2)

In [0]:
from tensorflow.python.keras import backend as K
from numpy import newaxis
def outer_product(x, y):
    """
    inputs: list of two tensors (of equal dimensions, 
        for which you need to compute the outer product
    """
    batchSize = K.shape(x)[0]
    outerProduct = x[:,:, newaxis] * y[:,newaxis,:]
    outerProduct = K.reshape(outerProduct, (batchSize, -1))
    # returns a flattened batch-wise set of tensors
    return outerProduct

In [51]:
from tensorflow.python.keras import backend as K
inputX = Input(shape=(400, ))
inputY = Input(shape=(400, ))
bilinearProduct = outer_product([inputX, inputY])

ValueError: ignored

In [49]:
print(bilinearProduct.shape)

(?, ?)
